In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/IEProject'

/content/drive/MyDrive/IEProject


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 데이터 불러오기
file_path = "./dataset/dart_merged_data.csv"   # 경로는 환경에 맞게 수정
df = pd.read_csv(file_path)

# 데이터 확인
print(df.head())
print(df.columns)

  corp_name  corp_code  year quarter report_date          자산총계          부채총계  \
0     DL이앤씨    1524093  2015      Q4  2015-12-31  1.206489e+13  7.259125e+12   
1     DL이앤씨    1524093  2016      Q1  2016-03-31  1.233208e+13  7.389715e+12   
2     DL이앤씨    1524093  2016      Q2  2016-06-30  1.237828e+13  7.324959e+12   
3     DL이앤씨    1524093  2016      Q3  2016-09-30  1.218542e+13  7.042494e+12   
4     DL이앤씨    1524093  2016      Q4  2016-12-31  1.239151e+13  7.246135e+12   

           자본총계           매출액          영업이익         분기순이익  
0  4.805769e+12           NaN           NaN           NaN  
1  4.942363e+12  2.253709e+12  9.077587e+10  3.103707e+10  
2  5.053324e+12  2.563786e+12  1.361732e+11  1.197966e+11  
3  5.142928e+12  2.457364e+12  1.306544e+11  1.090875e+11  
4  5.145374e+12  2.578911e+12  6.178428e+10  3.327409e+10  
Index(['corp_name', 'corp_code', 'year', 'quarter', 'report_date', '자산총계',
       '부채총계', '자본총계', '매출액', '영업이익', '분기순이익'],
      dtype='object')


In [5]:
import numpy as np
import pandas as pd

# 1단계: 불필요한 컬럼 제거 (corp_code, report_date)
print("=== 불필요한 컬럼 제거 ===")

print("제거 전 컬럼:")
print(df.columns.tolist())

# 제거할 컬럼들 (corp_name은 유지)
columns_to_drop = ['corp_code', 'report_date']

# 실제 존재하는 컬럼만 제거
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
print(f"\n제거할 컬럼: {existing_columns_to_drop}")

# 컬럼 제거
dart_processed = df.drop(columns=existing_columns_to_drop)

print(f"\n제거 후 컬럼:")
print(dart_processed.columns.tolist())
print(f"\n데이터 형태: {dart_processed.shape}")

# 결과 확인
print("\n처리된 데이터 샘플:")
print(dart_processed.head())

# 기업명 확인
print(f"\n포함된 기업: {dart_processed['corp_name'].unique()}")
print(f"기업별 데이터 개수:")
print(dart_processed['corp_name'].value_counts())


=== 불필요한 컬럼 제거 ===
제거 전 컬럼:
['corp_name', 'corp_code', 'year', 'quarter', 'report_date', '자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']

제거할 컬럼: ['corp_code', 'report_date']

제거 후 컬럼:
['corp_name', 'year', 'quarter', '자산총계', '부채총계', '자본총계', '매출액', '영업이익', '분기순이익']

데이터 형태: (1248, 9)

처리된 데이터 샘플:
  corp_name  year quarter          자산총계          부채총계          자본총계  \
0     DL이앤씨  2015      Q4  1.206489e+13  7.259125e+12  4.805769e+12   
1     DL이앤씨  2016      Q1  1.233208e+13  7.389715e+12  4.942363e+12   
2     DL이앤씨  2016      Q2  1.237828e+13  7.324959e+12  5.053324e+12   
3     DL이앤씨  2016      Q3  1.218542e+13  7.042494e+12  5.142928e+12   
4     DL이앤씨  2016      Q4  1.239151e+13  7.246135e+12  5.145374e+12   

            매출액          영업이익         분기순이익  
0           NaN           NaN           NaN  
1  2.253709e+12  9.077587e+10  3.103707e+10  
2  2.563786e+12  1.361732e+11  1.197966e+11  
3  2.457364e+12  1.306544e+11  1.090875e+11  
4  2.578911e+12  6.178428e+10  3.327409e+10  

In [6]:
# ================================================================
# 2단계: DART 데이터 필터링
print("=== DART 데이터 필터링 ===")

print(f"필터링 전 데이터 형태: {dart_processed.shape}")
print(f"필터링 전 기업별 데이터 개수:")
print(dart_processed['corp_name'].value_counts().head(10))

# 필터링 조건 적용
filter_conditions = []

# 예시: 대원, 자이에스앤디, 효성중공업, HDC현대산업개발
if '대원' in dart_processed['corp_name'].values:
    before = len(dart_processed[dart_processed['corp_name'] == '대원'])
    dart_processed = dart_processed[~((dart_processed['corp_name'] == '대원') & (dart_processed['year'] < 2018))]
    after = len(dart_processed[dart_processed['corp_name'] == '대원'])
    filter_conditions.append(f"대원 18년 Q1부터: {before - after}개 제거")

if '자이에스앤디' in dart_processed['corp_name'].values:
    before = len(dart_processed[dart_processed['corp_name'] == '자이에스앤디'])
    dart_processed = dart_processed[~((dart_processed['corp_name'] == '자이에스앤디') & (dart_processed['year'] < 2020))]
    after = len(dart_processed[dart_processed['corp_name'] == '자이에스앤디'])
    filter_conditions.append(f"자이에스앤디 20년 Q1부터: {before - after}개 제거")

if '효성중공업' in dart_processed['corp_name'].values:
    before = len(dart_processed[dart_processed['corp_name'] == '효성중공업'])
    dart_processed = dart_processed[~((dart_processed['corp_name'] == '효성중공업') & (
        (dart_processed['year'] < 2018) |
        ((dart_processed['year'] == 2018) & (dart_processed['quarter'].isin(['Q1','Q2'])))))]
    after = len(dart_processed[dart_processed['corp_name'] == '효성중공업'])
    filter_conditions.append(f"효성중공업 18년 Q3부터: {before - after}개 제거")

if 'HDC현대산업개발' in dart_processed['corp_name'].values:
    before = len(dart_processed[dart_processed['corp_name'] == 'HDC현대산업개발'])
    dart_processed = dart_processed[~((dart_processed['corp_name'] == 'HDC현대산업개발') & (
        (dart_processed['year'] < 2018) |
        ((dart_processed['year'] == 2018) & (dart_processed['quarter'].isin(['Q1','Q2'])))))]
    after = len(dart_processed[dart_processed['corp_name'] == 'HDC현대산업개발'])
    filter_conditions.append(f"HDC현대산업개발 18년 Q3부터: {before - after}개 제거")

print("\n=== 필터링 결과 요약 ===")
for cond in filter_conditions:
    print(f"  {cond}")

print(f"\n필터링 후 데이터 형태: {dart_processed.shape}")
print(f"필터링 후 기업별 데이터 개수:")
print(dart_processed['corp_name'].value_counts().head(10))

# 정렬
dart_processed = dart_processed.sort_values(['corp_name','year','quarter']).reset_index(drop=True)
print("필터링 및 정렬 완료")


=== DART 데이터 필터링 ===
필터링 전 데이터 형태: (1248, 9)
필터링 전 기업별 데이터 개수:
corp_name
DL이앤씨        39
GS건설         39
HDC현대산업개발    39
HJ중공업        39
HL D&I       39
HS화성         39
KCC건설        39
경동인베스트       39
계룡건설         39
금호건설         39
Name: count, dtype: int64

=== 필터링 결과 요약 ===
  대원 18년 Q1부터: 9개 제거
  자이에스앤디 20년 Q1부터: 17개 제거
  효성중공업 18년 Q3부터: 11개 제거
  HDC현대산업개발 18년 Q3부터: 11개 제거

필터링 후 데이터 형태: (1200, 9)
필터링 후 기업별 데이터 개수:
corp_name
DL이앤씨     39
GS건설      39
HJ중공업     39
HL D&I    39
KCC건설     39
HS화성      39
경동인베스트    39
계룡건설      39
대우건설      39
금호건설      39
Name: count, dtype: int64
필터링 및 정렬 완료


In [7]:
# ================================================================
# 3단계: 파생변수 생성
print("=== 파생변수 생성 ===")

dart_derived = dart_processed.copy()

# 주요 재무 지표 확인
required_cols = ['자산총계','부채총계','자본총계','매출액','영업이익','분기순이익']
missing = [c for c in required_cols if c not in dart_derived.columns]

if missing:
    print(f"필요한 컬럼이 없습니다: {missing}")
else:
    derived_list = []
    for corp in dart_derived['corp_name'].unique():
        corp_data = dart_derived[dart_derived['corp_name']==corp].copy()
        corp_data = corp_data.sort_values(['year','quarter']).reset_index(drop=True)

        corp_data['부채비율'] = (corp_data['부채총계']/corp_data['자본총계'])*100
        corp_data['자기자본비율'] = (corp_data['자본총계']/corp_data['자산총계'])*100
        corp_data['ROA'] = (corp_data['분기순이익']/corp_data['자산총계'])*100
        corp_data['ROE'] = np.where(
          corp_data['자본총계'] != 0,
          (corp_data['분기순이익'] / corp_data['자본총계']) * 100,
          np.nan)

        corp_data['매출액성장률'] = corp_data['매출액'].pct_change()*100
        corp_data['영업이익성장률'] = corp_data['영업이익'].pct_change()*100
        corp_data['순이익성장률'] = corp_data['분기순이익'].pct_change()*100

        derived_list.append(corp_data)

    dart_derived = pd.concat(derived_list, ignore_index=True)
    print("파생변수 생성 완료")

=== 파생변수 생성 ===
파생변수 생성 완료


In [8]:
# ================================================================
# 4단계: 최종 정리 (2015 Q4, 2016 Q1 제외)
print("=== 최종 데이터 정리 ===")

before = len(dart_derived)
dart_final = dart_derived[~((dart_derived['year']==2015)&(dart_derived['quarter']=='Q4'))]
dart_final = dart_final[~((dart_final['year']==2016)&(dart_final['quarter']=='Q1'))]
after = len(dart_final)

print(f"제거된 데이터: {before-after}개")
print(f"최종 데이터 형태: {dart_final.shape}")

=== 최종 데이터 정리 ===
제거된 데이터: 56개
최종 데이터 형태: (1144, 16)


In [9]:
# 점검할 파생변수 목록
derived_vars = ['ROE', 'ROA', '부채비율', '자기자본비율',
                '매출액성장률', '영업이익성장률', '순이익성장률']

# 파생변수별 결측값 개수
null_counts = dart_final[derived_vars].isnull().sum()
print("=== 파생변수별 결측값 개수 ===")
print(null_counts)

# 자본총계 0 이하 → ROE NaN 원인 확인
roe_issue = dart_final[dart_final['자본총계'] <= 0][['corp_name','year','quarter','자본총계','ROE']]

# 자산총계 0 이하 → ROA NaN 원인 확인
roa_issue = dart_final[dart_final['자산총계'] <= 0][['corp_name','year','quarter','자산총계','ROA']]

print("\n=== ROE NaN 원인 (자본총계 <= 0) ===")
print(roe_issue.head(10))

print("\n=== ROA NaN 원인 (자산총계 <= 0) ===")
print(roa_issue.head(10))

=== 파생변수별 결측값 개수 ===
ROE        0
ROA        0
부채비율       0
자기자본비율     0
매출액성장률     4
영업이익성장률    4
순이익성장률     4
dtype: int64

=== ROE NaN 원인 (자본총계 <= 0) ===
     corp_name  year quarter          자본총계         ROE
118      HJ중공업  2018      Q4 -7.081860e+11  170.946051
641       삼부토건  2025      Q1 -2.201192e+10  134.733722
642       삼부토건  2025      Q2 -7.195520e+10   68.028536
942       진흥기업  2016      Q4 -1.000871e+10  692.653406
1048      태영건설  2023      Q4 -4.402202e+11  348.298110
1049      태영건설  2024      Q1 -5.807041e+11    3.063209

=== ROA NaN 원인 (자산총계 <= 0) ===
Empty DataFrame
Columns: [corp_name, year, quarter, 자산총계, ROA]
Index: []


In [10]:
import numpy as np
import pandas as pd

dfn = dart_final.copy()

# 성장률 ↔ 원천 컬럼 매핑
rate_map = {
    "매출액성장률": "매출액",
    "영업이익성장률": "영업이익",
    "순이익성장률": "분기순이익",
}

reports = []

for rate_col, base_col in rate_map.items():
    # 전분기 값 붙이기
    dfn[f"{base_col}_전분기"] = dfn.groupby("corp_name")[base_col].shift(1)

    # 원인 분류
    cond_first   = dfn.groupby("corp_name").cumcount() == 0
    cond_prev0   = dfn[f"{base_col}_전분기"] == 0
    cond_prevnan = dfn[f"{base_col}_전분기"].isna()

    reason = np.select(
        [cond_first, cond_prev0, cond_prevnan],
        ["첫 분기(이전 없음)", "전분기=0", "전분기=NaN"],
        default="기타"
    )

    mask_nan = dfn[rate_col].isna()
    tmp = dfn.loc[mask_nan, ["corp_name","year","quarter", rate_col, base_col, f"{base_col}_전분기"]].copy()
    tmp["지표"] = rate_col
    tmp["원인"] = reason[mask_nan]

    reports.append(tmp)

# 성장률 NaN 상세 리포트
null_detail = pd.concat(reports, ignore_index=True).sort_values(["지표","corp_name","year","quarter"])
print("=== 성장률 NaN 상세 리포트 ===")
print(null_detail.to_string(index=False))

# 요약 통계(원인별 집계)
summary = null_detail.groupby(["지표","원인"], dropna=False).size().reset_index(name="건수").sort_values(["지표","건수"], ascending=[True, False])
print("\n=== 성장률 NaN 원인 요약 ===")
print(summary.to_string(index=False))


=== 성장률 NaN 상세 리포트 ===
corp_name  year quarter  매출액성장률          매출액  매출액_전분기      지표          원인  영업이익성장률         영업이익  영업이익_전분기  순이익성장률        분기순이익  분기순이익_전분기
HDC현대산업개발  2018      Q3     NaN 9.394877e+11      NaN  매출액성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN          NaN        NaN
       대원  2018      Q1     NaN 8.216205e+10      NaN  매출액성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN          NaN        NaN
   자이에스앤디  2020      Q1     NaN 6.698207e+10      NaN  매출액성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN          NaN        NaN
    효성중공업  2018      Q3     NaN 8.128184e+11      NaN  매출액성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN          NaN        NaN
HDC현대산업개발  2018      Q3     NaN          NaN      NaN  순이익성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN 8.552406e+10        NaN
       대원  2018      Q1     NaN          NaN      NaN  순이익성장률 첫 분기(이전 없음)      NaN          NaN       NaN     NaN 1.039076e+10        NaN
   자이에스앤디  

In [ ]:
dart_final

,corp_name,year,quarter,자산총계,부채총계,자본총계,매출액,영업이익,분기순이익,부채비율,자기자본비율,ROA,ROE,매출액성장률,영업이익성장률,순이익성장률
2,DL이앤씨,2016,Q2,1.237828e+13,7.324959e+12,5.053324e+12,2.563786e+12,1.361732e+11,1.197966e+11,144.953291,40.824110,0.967797,2.370650,13.758516,50.010346,285.979110
3,DL이앤씨,2016,Q3,1.218542e+13,7.042494e+12,5.142928e+12,2.457364e+12,1.306544e+11,1.090875e+11,136.935499,42.205579,0.895230,2.121117,-4.150984,-4.052776,-8.939388
4,DL이앤씨,2016,Q4,1.239151e+13,7.246135e+12,5.145374e+12,2.578911e+12,6.178428e+10,3.327409e+10,140.828150,41.523385,0.268523,0.646680,4.946246,-52.711673,-69.497809
5,DL이앤씨,2017,Q1,1.281212e+13,7.563966e+12,5.248157e+12,2.511359e+12,1.139837e+11,1.493461e+11,144.126142,40.962430,1.165663,2.845688,-2.619402,84.486574,348.836136
6,DL이앤씨,2017,Q2,1.320628e+13,7.818788e+12,5.387496e+12,3.106287e+12,1.430366e+11,1.045337e+11,145.128425,40.794942,0.791545,1.940302,23.689504,25.488610,-30.005759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,효성중공업,2024,Q2,5.066724e+12,3.811208e+12,1.255516e+12,1.193776e+12,6.265961e+10,2.820423e+10,303.557162,24.779637,0.556656,2.246426,21.256133,11.562713,7.529980
1196,효성중공업,2024,Q3,5.100602e+12,3.774763e+12,1.325839e+12,1.145185e+12,1.114241e+11,7.241692e+10,284.707478,25.993776,1.419772,5.461969,-4.070423,77.824419,156.759104
1197,효성중공업,2024,Q4,6.218813e+12,4.163194e+12,2.055618e+12,1.571511e+12,1.322304e+11,9.607912e+10,202.527582,33.054837,1.544975,4.673976,37.227736,18.673107,32.674949
1198,효성중공업,2025,Q1,6.689856e+12,4.574591e+12,2.115265e+12,1.076135e+12,1.023871e+11,1.036293e+11,216.265578,31.618996,1.549051,4.899115,-31.522277,-22.569192,7.858288


In [11]:
# 성장률 관련 컬럼
growth_cols = ['매출액성장률', '영업이익성장률', '순이익성장률']

# NaN이 있는 행 찾기
growth_null_rows = dart_final[dart_final[growth_cols].isnull().any(axis=1)]
print("=== 성장률 NaN 행 ===")
print(growth_null_rows[['corp_name','year','quarter'] + growth_cols])

# NaN이 포함된 행 제거
dart_final = dart_final.dropna(subset=growth_cols).reset_index(drop=True)

print(f"\n제거된 행 개수: {len(growth_null_rows)}")
print(f"제거 후 데이터 형태: {dart_final.shape}")

=== 성장률 NaN 행 ===
      corp_name  year quarter  매출액성장률  영업이익성장률  순이익성장률
78    HDC현대산업개발  2018      Q3     NaN      NaN     NaN
496          대원  2018      Q1     NaN      NaN     NaN
916      자이에스앤디  2020      Q1     NaN      NaN     NaN
1172      효성중공업  2018      Q3     NaN      NaN     NaN

제거된 행 개수: 4
제거 후 데이터 형태: (1140, 16)


In [ ]:
# 최종 데이터 저장
save_path = "./dataset/dart_final.csv"   # 원하는 경로로 수정 가능
dart_final.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"최종 데이터 저장 완료: {save_path}")

최종 데이터 저장 완료: ./dataset/dart_final.csv


In [17]:
import pandas as pd
import numpy as np

df = dart_final.copy()

# 분기 정렬
qcat = pd.CategoricalDtype(categories=['Q1','Q2','Q3','Q4'], ordered=True)
df['quarter'] = df['quarter'].astype(qcat)
df = df.sort_values(['corp_name','year','quarter']).reset_index(drop=True)

# 숫자형 보정
num_cols = ['자산총계','부채총계','자본총계','매출액','영업이익','분기순이익','부채비율','영업이익성장률']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

g = df.groupby('corp_name', group_keys=False)

# 1) 완전자본잠식
df['완전자본잠식'] = (df['자본총계'] < 0).astype(int)

# 2) 8분기 연속 순이익 감소
df['8분기 연속 순이익 감소'] = g['분기순이익'].transform(
    lambda s: (s.diff() < 0).astype('int8').rolling(8, min_periods=8).sum()
).eq(8).astype(int)

# 3) 부채비율 300% 초과
df['부채비율 300% 초과'] = (df['부채비율'] >= 300).astype(int)

# 4) 4분기 연속 영업손실
df['4분기 연속 영업손실'] = g['영업이익'].transform(
    lambda s: (s < 0).astype('int8').rolling(4, min_periods=4).sum()
).eq(4).astype(int)

# 5) 8분기 연속 영업이익 감소
df['8분기 연속 영업이익 감소'] = g['영업이익성장률'].transform(
    lambda s: (s < 0).astype('int8').rolling(8, min_periods=8).sum()
).eq(8).astype(int)

risk_cols = ['완전자본잠식','8분기 연속 순이익 감소','부채비율 300% 초과','4분기 연속 영업손실','8분기 연속 영업이익 감소']
df['총위험점수'] = df[risk_cols].sum(axis=1)

print("=== 전체 위험 지표 발생 건수(분기 누적) ===")
print(df[risk_cols + ['총위험점수']].sum())

print("\n=== 총 위험 점수 분포 ===")
print(df['총위험점수'].value_counts().sort_index())

print("\n=== 기업별 누적 발생 건수(분기 합) ===")
print(df.groupby('corp_name')[risk_cols + ['총위험점수']].sum()
        .sort_values('총위험점수', ascending=False).head(32))

print("\n=== 기업별 ever 발생 여부(한 번이라도 발생) ===")
ever = df.groupby('corp_name')[risk_cols].max()
ever = ever.assign(총위험점수_ever=lambda x: x.sum(axis=1))
print(ever.sort_values('총위험점수_ever', ascending=False).head(32))

dart_with_risk = df.copy()


=== 전체 위험 지표 발생 건수(분기 누적) ===
완전자본잠식              6
8분기 연속 순이익 감소       0
부채비율 300% 초과      201
4분기 연속 영업손실        30
8분기 연속 영업이익 감소     13
총위험점수             250
dtype: int64

=== 총 위험 점수 분포 ===
총위험점수
0    900
1    230
2     10
Name: count, dtype: int64

=== 기업별 누적 발생 건수(분기 합) ===
           완전자본잠식  8분기 연속 순이익 감소  부채비율 300% 초과  4분기 연속 영업손실  8분기 연속 영업이익 감소  \
corp_name                                                                     
HJ중공업           1              0            36            1               0   
코오롱글로벌          0              0            33            0               0   
HL D&I          0              0            26            0               0   
삼부토건            2              0             7           15               1   
태영건설            2              0            18            0               0   
효성중공업           0              0            14            0               0   
진흥기업            1              0            11            0               0   
계룡건설   